In [84]:
import pandas as pd
import numpy as np
import scipy
import math

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

In [94]:
def write_answer(assignment_N, answer):
    with open("gene_expression{}.txt".format(assignment_N), "w") as fout:
        fout.write("{0}".format(answer))

In [2]:
data = pd.read_csv('high_throughput_sequencing.csv')

In [3]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [4]:
gens = data.columns[2:]

In [5]:
data_normal = data[data['Diagnosis']=='normal'][gens]
data_en = data[data['Diagnosis']=='early neoplasia'][gens]
data_cancer = data[data['Diagnosis']=='cancer'][gens]

In [32]:
def calc_data(data1,data2):
    results = []
    for gen in gens:
        statistics, p =scipy.stats.ttest_ind(data1[gen], data2[gen], equal_var = False)
        results.append([gen, np.mean(data1[gen]), np.mean(data2[gen]), statistics, p])
    return pd.DataFrame(results, columns=['Gen','Control','Treatment','Stat','p'])

In [33]:
normal_vs_en = calc_data(data_normal, data_en)

In [34]:
normal_vs_en.head()

,Gen,Control,Treatment,Stat,p
0,LOC643837,2.681277,2.510894,0.400289,0.690766
1,LOC100130417,4.368497,8.721781,-4.608766,0.000032
2,SAMD11,15.159566,18.531325,-1.929277,0.060273
3,NOC2L,15.374351,15.071854,0.220542,0.826429
4,KLHL17,21.459886,24.152469,-2.013201,0.049876


In [55]:
normal_vs_en.shape

(15748, 11)

In [95]:
write_answer(1, np.sum(normal_vs_en.p < 0.05))

In [35]:
en_vs_cancer = calc_data(data_en, data_cancer)

In [96]:
write_answer(2, np.sum(en_vs_cancer.p < 0.05))

In [38]:
def fold_change(C,T):
    if T>=C:
        return float(T)/C
    else:
        return -float(C)/T

In [39]:
def correct_p(data, alpha, method):
    reject, p_corrected, a1, a2 = multipletests(data.p, 
                                            alpha = alpha, 
                                            method = method)
    data[method+'_p_corrected'] = p_corrected
    data[method+'_reject'] = reject
    
    return data

In [20]:
# Метод Холма

In [40]:
normal_vs_en = correct_p(normal_vs_en, alpha=0.025, method='holm')
en_vs_cancer = correct_p(en_vs_cancer, alpha=0.025, method='holm')

In [41]:
# Метод Бенджамини-Хохберга

In [65]:
normal_vs_en = correct_p(normal_vs_en, alpha=0.025, method='fdr_bh')
en_vs_cancer = correct_p(en_vs_cancer, alpha=0.025, method='fdr_bh')

In [43]:
normal_vs_en.head()

,Gen,Control,Treatment,Stat,p,holm_p_corrected,holm_reject,fdr_bh_p_corrected,fdr_bh_reject
0,LOC643837,2.681277,2.510894,0.400289,0.690766,1.000000,False,0.966511,False
1,LOC100130417,4.368497,8.721781,-4.608766,0.000032,0.500174,False,0.035698,False
2,SAMD11,15.159566,18.531325,-1.929277,0.060273,1.000000,False,0.536103,False
3,NOC2L,15.374351,15.071854,0.220542,0.826429,1.000000,False,0.980777,False
4,KLHL17,21.459886,24.152469,-2.013201,0.049876,1.000000,False,0.499016,False


In [83]:
en_vs_cancer.head()

,Gen,Control,Treatment,Stat,p,holm_p_corrected,holm_reject,fdr_bh_p_corrected,fdr_bh_reject,holm_fold_rejected,fdr_bh_fold_rejected
0,LOC643837,2.510894,2.186060,0.824849,0.413735,1.0,False,0.675195,False,False,False
1,LOC100130417,8.721781,8.190456,0.452236,0.653429,1.0,False,0.836406,False,False,False
2,SAMD11,18.531325,23.692614,-1.817266,0.079556,1.0,False,0.288873,False,False,False
3,NOC2L,15.071854,16.468034,-1.075985,0.287581,1.0,False,0.563007,False,False,False
4,KLHL17,24.152469,25.035813,-0.740329,0.463292,1.0,False,0.712214,False,False,False


In [87]:
def check_fold_change(x):
    fold_level = 1.5
    if x[2] and abs(fold_change(x[0],x[1]))>fold_level:
        return True
    return False

In [88]:
normal_vs_en['holm_fold_rejected'] = normal_vs_en[['Control','Treatment','holm_reject']].apply(check_fold_change,axis=1)
en_vs_cancer['holm_fold_rejected'] = en_vs_cancer[['Control','Treatment','holm_reject']].apply(check_fold_change,axis=1)

In [89]:
normal_vs_en['fdr_bh_fold_rejected'] = normal_vs_en[['Control','Treatment','fdr_bh_reject']].apply(check_fold_change,axis=1)
en_vs_cancer['fdr_bh_fold_rejected'] = en_vs_cancer[['Control','Treatment','fdr_bh_reject']].apply(check_fold_change,axis=1)

In [98]:
write_answer(3, np.sum(normal_vs_en['holm_fold_rejected']))

In [99]:
write_answer(4, np.sum(en_vs_cancer['holm_fold_rejected']))

In [100]:
write_answer(5, np.sum(normal_vs_en['fdr_bh_fold_rejected']))

In [101]:
write_answer(6, np.sum(en_vs_cancer['fdr_bh_fold_rejected']))